# (MBTI 별 Data 모으기: 생략) MBTI file 불러오기

In [93]:
import os
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import re
import urllib.request
from collections import Counter

In [94]:
mbti=pd.read_csv('.\data\\training\\mbti.csv')

# UnderSampling

In [95]:
from imblearn.under_sampling import RandomUnderSampler

X_data = mbti['posts']
X = X_data.values.reshape(-1, 1)

y_data = mbti['type']
print(y_data)
y = y_data.values.reshape(-1, 1)
print(y)

X_resampled, y_resampled = RandomUnderSampler(random_state=0).fit_sample(X, y)
#format(Counter(y_resampled))
X_resampled #ndarray
Counter(y_resampled)


0       ENFJ
1       ENFJ
2       ENFJ
3       ENFJ
4       ENFJ
        ... 
8670    ISTP
8671    ISTP
8672    ISTP
8673    ISTP
8674    ISTP
Name: type, Length: 8675, dtype: object
[['ENFJ']
 ['ENFJ']
 ['ENFJ']
 ...
 ['ISTP']
 ['ISTP']
 ['ISTP']]


Counter({'ENFJ': 39,
         'ENFP': 39,
         'ENTJ': 39,
         'ENTP': 39,
         'ESFJ': 39,
         'ESFP': 39,
         'ESTJ': 39,
         'ESTP': 39,
         'INFJ': 39,
         'INFP': 39,
         'INTJ': 39,
         'INTP': 39,
         'ISFJ': 39,
         'ISFP': 39,
         'ISTJ': 39,
         'ISTP': 39})

# Type value 

In [96]:
b_Types = {'I':0, 'E':1, 'N':0, "S":1, 'F':0, 'T':1, 'J':0, 'P':1}
b_Types_list =[{0:'I', 1:'E'}, {0:'N', 1:'S'}, {0:'F', 1:'T'}, {0:'J', 1:'P'}]

def translate_personality(personality):
    return[b_Types[i] for i in personality]

def translate_back(personality):
    s = ""
    for i, j in enumerate(personality):
        s += b_Types_list[i][j]
    return s

d = y[0]
print(d)
list_personality_b = np.array([translate_personality(p) for p in d])
print(list_personality_b)

['ENFJ']
[[1 0 0 0]]


# tokenize and stemming

In [97]:
X_data = X_resampled.ravel()
y_data = y_resampled


print("normalize 시작")
normalized_text = []
for sentence in X_data: #for sentence in X_data.ravel()
    rm_urls = re.sub(r'http[s]?://\S+', '', sentence)
    clean_sentence = re.sub('[^A-Za-z\s]+', '', rm_urls.lower())
    normalized_text.append(clean_sentence)
X_data = normalized_text
print("normalize 완료")

normalize 시작
normalize 완료


In [98]:
import nltk
import gc
# nltk.download('punkt')
# nltk.download('wordnet')
from nltk import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [99]:
ps=PorterStemmer()

# print("stopwords 제거 시작")
stop_words = set(stopwords.words('english'))
#print("stopwords 제거 완료")

print("stemming 시작")
stemData=[]
for sentence in X_data:
    tokenData = nltk.word_tokenize(sentence)
    tempData = []
    for word in tokenData:
        if word not in stop_words:
            tempData.append(ps.stem(word))
    stemData.append(tempData)
print("stemming 완료")
print(stemData[0])
stemData = np.array(stemData)
#'list' object has no attribute 'lower' 문제 발생
#2차원 리스트인 stemData를 1차원 리스트로
#flat_stem = [item for sublist in stemData for item in sublist]
del [[mbti]]
gc.collect()

stemming 시작
stemming 완료
['want', 'know', 'feel', 'intp', 'actual', 'intp', 'idea', 'intp', 'havent', 'met', 'anyten', 'million', 'hug', 'mani', 'amaz', 'convers', 'held', 'absenc', 'vow', 'never', 'leav', 'realli', 'lot', 'written', 'recent', 'terriblyfacebook', 'way', 'emphas', 'other', 'error', 'cognit', 'becom', 'obviou', 'lead', 'wonder', 'im', 'friend', 'mani', 'peopl', 'rememb', 'peopl', 'moremmmmhm', 'ive', 'wonder', 'late', 'ei', 'think', 'definit', 'much', 'needwant', 'compani', 'around', 'regardless', 'time', 'togeth', 'spenthahah', 'love', 'think', 'that', 'go', 'topic', 'excel', 'convers', 'thank', 'particip', 'indulg', 'interest', 'enlighteningsomeon', 'creat', 'chitchat', 'thread', 'enfj', 'forum', 'enfj', 'vigilantli', 'await', 'type', 'wander', 'start', 'conversationswa', 'enfp', 'ive', 'caught', 'mani', 'text', 'look', 'around', 'shini', 'object', 'give', 'midsyl', 'mm', 'hmm', 'alway', 'mani', 'andi', 'tire', 'extravert', 'energysap', 'sentiment', 'arent', 'batshit', 

2740

# Vectorization

In [100]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [101]:
count_vectorizer = CountVectorizer(
    analyzer="word",
    max_features=1500,
    tokenizer=None,
    stop_words=None,
    max_df=0.7,
    min_df=0.1
)

#X_ct = count_vectorizer.fit_transform(stemData)

def dummy_fun(doc):
    return doc

tfidf=TfidfVectorizer(
    analyzer='word',
    tokenizer=dummy_fun,
    preprocessor=dummy_fun,
    token_pattern=None
)

tfidf.fit(stemData)
print(tfidf.transform(stemData).toarray())
X = tfidf.transform(stemData)
print(X)
y = np.array(y_data)

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.05858548 0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.04542902 0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]
  (0, 38616)	0.031996512659156894
  (0, 38578)	0.03286039560761366
  (0, 38565)	0.057208490852473474
  (0, 38556)	0.06369299789656549
  (0, 38482)	0.07370640937171992
  (0, 38432)	0.023110327077128505
  (0, 38411)	0.056118144533601276
  (0, 38346)	0.030911604753450596
  (0, 38304)	0.050319331085647706
  (0, 38051)	0.03852987969548263
  (0, 37982)	0.021758208967633117
  (0, 37964)	0.035121536565144516
  (0, 37804)	0.03987971435535688
  (0, 37774)	0.02453157411311739
  (0, 37752)	0.04127966139118793
  (0, 37742)	0.028172244380434894
  (0, 37325)	0.0661

# XGBoost_Undersampling

In [102]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.3, random_state=1234)


# Model Building & Evaluation

In [103]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from matplotlib import pyplot

In [104]:
print("Model building Start")
model = XGBClassifier()
model.fit(X_train, y_train, verbose=1)
#model.get_booster().get_score(importance_type="gain") #변수의 중요도

Model building Start


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [105]:
import xgboost as xgb
#test_hp = xgb.DMatrix(data=VHX_data, label=hy)
preds = model.predict(X_test)
print("Model building End")

Model building End


In [106]:
import joblib

joblib.dump(model, 'XGBoost_undersample.pkl')

['XGBoost_undersample.pkl']

In [107]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

print("Evaluation Start")
# labels과 guesses
labels = preds
guesses = y_test

print(accuracy_score(labels, guesses))
print(recall_score(labels, guesses, average=None))
print(precision_score(labels, guesses, average=None))
print(f1_score(labels, guesses, average=None))

print("Evaluation END")
# https://eunsukimme.github.io/ml/2019/10/21/Accuracy-Recall-Precision-F1-score/

Evaluation Start
0.4946808510638298
[0.71428571 0.4        0.27272727 0.23529412 0.81818182 0.41666667
 0.54545455 1.         0.33333333 0.7        0.375      0.16666667
 0.5        0.46666667 0.61111111 0.58333333]
[0.625      0.30769231 0.375      0.4        0.5625     0.45454545
 0.4        0.61538462 0.25       0.53846154 0.33333333 0.2
 0.66666667 0.7        0.61111111 0.63636364]
[0.66666667 0.34782609 0.31578947 0.2962963  0.66666667 0.43478261
 0.46153846 0.76190476 0.28571429 0.60869565 0.35294118 0.18181818
 0.57142857 0.56       0.61111111 0.60869565]
Evaluation END


# Harry Potter

In [108]:
import nltk
import numpy as np
import pandas as pd

from nltk import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

import re
import glob   

'''HP data Normalization and Stemming'''
# bag_of_words = {} #{등장인물 : 해당인물의 stemData, ~}
h_normalized_text = []
h_stemData = []

ps=PorterStemmer()
stop_words = set(stopwords.words('english'))
input_file = r'.\data\\test\\'

allHPfile_list = glob.glob(os.path.join(input_file, 'HP_*'))
print(allHPfile_list)

harryPotter={}
for file in allHPfile_list:
    df = pd.read_csv(file, sep=',', encoding='iso-8859-1')
    harryPotter[df['Character'][0]] = df['Sentence'].ravel()

print("stemming 시작")
for element in harryPotter.keys():
    for sentence in harryPotter[element]:
        h_clean_sentence = re.sub('[^A-Za-z\s]+', '', sentence.lower())
        h_normalized_text.append(h_clean_sentence)
        for sentence in h_normalized_text:
            tokenData = nltk.word_tokenize(sentence)
            for word in tokenData:
                if word not in stop_words:
                    h_stemData.append(ps.stem(word))
    harryPotter[element] = h_stemData

print("stemming 완료")

['.\\data\\\\test\\HP_DUMBLEDORE.csv', '.\\data\\\\test\\HP_HAGRID.csv', '.\\data\\\\test\\HP_HARRY.csv', '.\\data\\\\test\\HP_HERMIONE.csv', '.\\data\\\\test\\HP_RON.csv']
stemming 시작
stemming 완료


In [109]:
'''HP data Vectorization'''
# import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

def dummy_fun(doc):
    return doc

tfidf=TfidfVectorizer(
    analyzer='word',
    tokenizer=dummy_fun,
    preprocessor=dummy_fun,
    token_pattern=None
)

stemData = []
hy=[]
for element in harryPotter.keys():
    stemData.append(harryPotter[element])
    hy.append(element)
HX_data = tfidf.fit_transform(stemData)
print(HX_data)
HY_data = np.array(hy)

'''for element in harryPotter.keys():
    tfidf.fit(harryPotter[element])
    script = tfidf.transform(harryPotter[element])
    X.append(script)
    y.append(element)

print(X.shape)'''

  (0, 1499)	7.338210456553937e-06
  (0, 89)	7.338210456553937e-06
  (0, 1154)	1.7122491065292518e-05
  (0, 169)	4.158319258713898e-05
  (0, 346)	0.00010273494639175512
  (0, 733)	7.093603441335472e-05
  (0, 1365)	7.093603441335472e-05
  (0, 36)	7.827424486990865e-05
  (0, 1372)	9.784280608738582e-05
  (0, 596)	0.00010028887623957047
  (0, 1495)	0.0001614406300441866
  (0, 1310)	0.00029597448841434213
  (0, 783)	0.00016388670019637126
  (0, 133)	0.00016388670019637126
  (0, 554)	0.00017367098080510983
  (0, 228)	0.00017611705095729448
  (0, 825)	0.00017611705095729448
  (0, 1441)	0.00018590133156603307
  (0, 1195)	0.000193239542022587
  (0, 857)	0.00019568561217477164
  (0, 1393)	0.00020057775247914094
  (0, 43)	0.0002030238226313256
  (0, 1467)	0.0002030238226313256
  (0, 1463)	0.0002030238226313256
  (0, 128)	0.0002030238226313256
  :	:
  (4, 443)	0.007325980105793013
  (4, 603)	0.06672390161129277
  (4, 416)	0.04377487144349642
  (4, 437)	0.02377824794938694
  (4, 962)	0.032383522744

'for element in harryPotter.keys():\n    tfidf.fit(harryPotter[element])\n    script = tfidf.transform(harryPotter[element])\n    X.append(script)\n    y.append(element)\n\nprint(X.shape)'

In [110]:
print(len(HX_data.toarray()[0]))
print(len(X.toarray()[0]))

1527
38863


In [111]:
'''HP data Undersample'''
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

'''HX_data, HY_data = RandomOverSampler(random_state=0,
sampling_strategy={'HARRY':38863,'RON':38863,'HERMIONE':38863,'DUMBLEDORE':38863,'HAGRID':38863}).fit_sample(HX_data, HY_data)'''
#format(Counter(y_resampled))
HX_data #ndarray
Counter(HY_data)

Counter({'DUMBLEDORE': 1, 'HAGRID': 1, 'HARRY': 1, 'HERMIONE': 1, 'RON': 1})

# Predict & Get Results

In [112]:
import joblib  
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from matplotlib import pyplot

xgb_model_loaded = joblib.load('XGBoost_undersample.pkl')
# xgb_model_loaded.get_xgb_params()    # Get xgboost specific parameters.
harrypotter_predict = xgb_model_loaded.predict(HX_data)
# print(harrypotter_predict)

ValueError: feature_names mismatch: ['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20', 'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29', 'f30', 'f31', 'f32', 'f33', 'f34', 'f35', 'f36', 'f37', 'f38', 'f39', 'f40', 'f41', 'f42', 'f43', 'f44', 'f45', 'f46', 'f47', 'f48', 'f49', 'f50', 'f51', 'f52', 'f53', 'f54', 'f55', 'f56', 'f57', 'f58', 'f59', 'f60', 'f61', 'f62', 'f63', 'f64', 'f65', 'f66', 'f67', 'f68', 'f69', 'f70', 'f71', 'f72', 'f73', 'f74', 'f75', 'f76', 'f77', 'f78', 'f79', 'f80', 'f81', 'f82', 'f83', 'f84', 'f85', 'f86', 'f87', 'f88', 'f89', 'f90', 'f91', 'f92', 'f93', 'f94', 'f95', 'f96', 'f97', 'f98', 'f99', 'f100', 'f101', 'f102', 'f103', 'f104', 'f105', 'f106', 'f107', 'f108', 'f109', 'f110', 'f111', 'f112', 'f113', 'f114', 'f115', 'f116', 'f117', 'f118', 'f119', 'f120', 'f121', 'f122', 'f123', 'f124', 'f125', 'f126', 'f127', 'f128', 'f129', 'f130', 'f131', 'f132', 'f133', '

In [ ]:
print(harrypotter_predict)

In [ ]:
cols_when_model_builds = xgb_model_loaded.get_booster().feature_names
len(cols_when_model_builds)